In [18]:
from ultralytics import YOLO

trained = YOLO('Phone-Detection.pt')

In [19]:
from playsound import playsound

In [20]:
from io import BytesIO
from PIL import Image
import base64

# Function to convert image to base64
def image_to_base64(image):
    # Convert OpenCV image (NumPy array) to PIL Image
    pil_image = Image.fromarray(image)

    # Save the image to a BytesIO object as PNG format
    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")

    # Get the base64 encoded string
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

In [21]:
import pygame
from threading import Thread

# Initialize the sound system using pygame
pygame.mixer.init()

# Load the sound file (ensure it's in the same directory or provide a full path)
camera_sound = pygame.mixer.Sound("camera.mp3")

In [22]:
# Function to play sound and prevent overlap
def play_sound():
    # Check if the sound is currently playing
    if not pygame.mixer.get_busy():
        camera_sound.play()

In [27]:
from ultralytics import YOLO
import cv2
import mss
import numpy as np
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# Open the webcam 
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

width  = cap.get(3)  # float `width`
height = cap.get(4)  # float `height`

cellphone_list = set()

with mss.mss() as sct:

    while True:

        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture image.")
            break

        # Run YOLO inference on the frame
        results = trained.track(frame, persist=True, verbose=False, conf=0.3)

        for result in results:  # Results as [x1, y1, x2, y2, confidence, class]
            for i in range(len(result.boxes.cls)):
                x1, y1, x2, y2 = result.boxes.xyxy[i]
                conf = result.boxes.conf[i].item()
                try:
                    if(result.boxes.id[i]):
                        id = result.boxes.id[i].item()
                except:
                    pass
                x1 = int(x1.item())
                y1 = int(y1.item())
                x2 = int(x2.item())
                y2 = int(y2.item())

                if(id not in cellphone_list):

                    # Play the camera sound in a separate thread (non-blocking)
                    sound_thread = Thread(target=play_sound)
                    sound_thread.start()

                    cellphone_list.add(id)

                    # Capture the screen region
                    cropped_frame = frame[int(max(0, y1-20)):int(min(height, y2 + 20)), int(max(0, x1-20)):int(min(width, x2+20))]

                    # Change colour code
                    cropped_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)

                    # Convert the captured image to base64
                    img_base64 = image_to_base64(cropped_frame)

                    response = requests.post(os.getenv('URL'), json={"args": {"crosswalk_name": "default_crosswalk", "storageId": "data:image/png;base64," + img_base64}, "path": "camera_data:addImage"})


        # Visualize the results on the frame
        annotated_frame = results[0].plot()  # YOLOv8 has built-in plotting for results

        # Display the resulting frame with annotations
        cv2.imshow('YOLO Webcam', annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


# When everything is done, release the capture and close the window
cap.release()
cv2.destroyAllWindows()




Error: Failed to capture image.
